In [48]:
import shutil
import torch
import cv2
import os

import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import albumentations as A
import pandas as pd
import numpy as np

from albumentations.pytorch.transforms import ToTensorV2
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import DatasetFolder
from tqdm import tqdm; tqdm.pandas();

In [49]:
# Устройство для обучения
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Current device:", device)

Current device: cuda


# Настройка параметров

In [50]:
# Директории с файлами
DATASET_DIR = './train_landmark_files'
CLASSES_DIR = './Dataset'

# Параметры загрузчиков
BACTH_SIZE  = 1
NUM_WORKERS = 0
PIN_MEMORY  = True

# Параметры оптимизатора
WEIGHT_DECAY  = 5e-5
LEARNING_RATE = 5e-3

MEAN = [0.3395, 0.3431, 0.3020]
STD  = [0.3908, 0.3897, 0.3542]

# Параметры планировщика
FACTOR    = 0.1
PATIENCE  = 10
THRESHOLD = 1e-4

# Параметры модели
ENCODER         = 'resnet50'
ENCODER_WEIGHTS = None

# Параметры обучения
NUM_EPOCHS = 30

# Параметры датасета
ORIGINAL_SIZE = 1500
PADED_SIZE    = 1536
PATCH_SIZE    = 256 
VALID_PART = 0.1
TEST_PART  = 0.1

# Настройка аугментаций

In [51]:
train_transform = A.Compose([
        A.RandomBrightnessContrast(brightness_limit = 0.1, contrast_limit = 0.1, p = 0.5),
        A.PadIfNeeded(PADED_SIZE, PADED_SIZE, border_mode = 0),
        A.Normalize(mean = MEAN, std = STD),        
        ToTensorV2()])

val_test_transform = A.Compose([
    A.PadIfNeeded(PADED_SIZE, PADED_SIZE, border_mode = 0),
    A.Normalize(mean = MEAN, std = STD),
    ToTensorV2()])

# Распределение файлов по классам

In [52]:
df = pd.read_csv('./train.csv')
df.head()

,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie


In [53]:
def path_handle(path, sign):

    if not os.path.exists(os.path.join(CLASSES_DIR, sign)):        
        os.mkdir(os.path.join(CLASSES_DIR, sign))

    shutil.move(path, os.path.join(CLASSES_DIR, sign))

In [54]:
if not os.path.exists(CLASSES_DIR):
    os.mkdir(CLASSES_DIR)

    [path_handle(path, sign) for path, _, _, sign in zip(df['path'], df['participant_id'], df['sequence_id'], df['sign'])]

# Разделение датасета

In [55]:
dataset = DatasetFolder(CLASSES_DIR)

# n_val   = len(dataset) * VALID_PART
# n_test  = len(dataset) * TEST_PART
# n_train = len(dataset) - n_val - n_test

# train_dataset, val_dataset, test_dataset = random_split(dataset, [n_train, n_val, n_test])

len(dataset)

TypeError: DatasetFolder.__init__() missing 1 required positional argument: 'loader'